# Frozen Lake

Using a genetic algorithm to find the way through a maze. This is mostly similar to the previous example. The largest differences are in the fitness calculation. We still associate a string of inputs (dna) to a fitness value which determines procreation.

In [1]:
import sys
import gym
import numpy as np

map_name can be eiter '4x4' or '8x8'

In [2]:
from gym.envs.registration import register

# Setup custom environment
register(
    id='FrozenLakeIsNotSlip-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={
        'map_name' : '4x4',
        'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.78)

In [3]:
env = gym.make('FrozenLakeIsNotSlip-v0')

# Global parameters
POPULATION_SIZE = 200
MUTATION_RATE = 0.02
DNA_LENGTH = 50
MAX_ITER = 100

In [4]:
class Individual():
    """
    Class containing info and functions on individuals in the
    population. Depending on value of dna, will create new
    dna or accept input.

    Parameters:
        genes (str): str with options to generate dna.
        target (str): str with target of evolution
        target_length (int): length of target
        dna (str): dna sequence of individual
    """

    def __init__(self,
                 env,
                 dna_length: int,
                 dna: str = None) -> None:

        self.env = env

        self.dna_length = dna_length

        if dna is None:
            self.dna = []
            for _ in range(self.dna_length):
                self.dna.append(self.env.action_space.sample())
        else:
            self.dna = dna

        self.fitness = self.calc_fitness()


    def calc_fitness(self) -> float:
        """
        Run environment with the current dna sequence. Each step the
        player is not done adds to the fitness score. When the finish
        is reached, start penalizing long pahts.
        """

        self.env.reset()
        counter = 0
        fitness = 0
        for i in range(self.dna_length):
            _, reward, done, _ = self.env.step(self.dna[i])
            counter += 1
            if done:
                if reward != 0.0:
                    fitness = reward - (counter * 0.001)
                else:
                    fitness = counter * 0.001
                break
        return fitness


    def mix_dna(self, dna_2: str, mutation_rate: float) -> str:
        """
        Given a second DNA sequence, mix two sequences and return
        offspring DNA. Based on mutation rate, mutate certain genes.
        """

        new_dna = []
        for i in range(self.dna_length):
            choice = np.random.uniform()
            if choice < mutation_rate:
                new_gene = self.env.action_space.sample()
            else:
                new_gene = np.random.choice([self.dna[i], dna_2[i]])
            new_dna.append(new_gene)
        return new_dna

In [5]:
from typing import List

class Population():
    """
    Control population during running the genetic algorithm. Has
    functions for creating a population, producing offspring and
    scoring the population.

    Parameters:
        genes (str): options to create dna from
        population_size (int): amount of individuals per generation
        dna_length (int): length of target
        mutation_rate (float): determines mutation during reproduction
    """

    def __init__(self,
                 env,
                 population_size: int,
                 mutation_rate: float,
                 dna_length: int) -> None:


        self.env = env

        self.population: List[Individual] = []
        self.population_size = population_size

        self.mutation_rate = mutation_rate
        self.dna_length = dna_length

        self.max_fitness: float = 0.0

    def init_population(self) -> None:
        """
        Create n individuals where n is the population
        size. Save these in self.population.
        """

        for _ in range(self.population_size):
            self.population.append(Individual(
                self.env,
                self.dna_length))


    def _create_p_list(self) -> None:
        """
        Create a list of fitness scores and scale it to have a sum
        of 1.
        """

        p_list = np.array([ind.fitness for ind in self.population])
        return p_list / np.sum(p_list)


    def create_offspring(self) -> None:
        """
        For population size, pick two individuals from the population
        based on the fitness score and create offspring, this will be
        the new population.
        """

        new_population = []
        p_list = self._create_p_list()
        for _ in range(self.population_size):
            # Select to individuals, chance is scaled to fitness score
            parents = np.random.choice(self.population, 2, p=p_list)
            # Create new dna from parents
            new_dna = parents[0].mix_dna(
                parents[1].dna, self.mutation_rate)
            new_population.append(Individual(
                self.env,
                self.dna_length,
                new_dna))
        self.population = new_population


    def calc_fitness(self):
        """
        Sort the population and extract the maximal fitness from
        the population
        """
        self.population = sorted(
            self.population,
            key=lambda x: x.fitness,
            reverse=True)
        self.max_fitness = self.population[0].fitness

In [10]:
from IPython.display import clear_output
from time import sleep

def run_dna(env, dna):
    """
    Given a environment and a dna sequence, run and show the sequence.
    """

    env.reset()
    env.render()
    sleep(.1)
    clear_output(wait=True)
    for i, s in enumerate(dna):
        _, _, done, _ = env.step(s)
        env.render()
        if done:
            print(f"\n {i} steps needed")
            break
        sleep(.1)
        clear_output(wait=True)
        

In [7]:
def main():
    """
    Control the genetic algorithm:

    First create a population. The population is passed all global
    variables. Then initiate the population and calculate the fitness.
    Start the loop, print the current fitness and best dna sequence,
    when max fitness is reached, break out of loop and quit.
    """

    print(f"""
Population Size: {POPULATION_SIZE}
Mutation_rate: {MUTATION_RATE}
DNA_length: {DNA_LENGTH}
Max_iter : {MAX_ITER}
""")

    population = Population(env,
                            POPULATION_SIZE,
                            MUTATION_RATE,
                            DNA_LENGTH)
    population.init_population()
    population.calc_fitness()
    max_fitness = population.max_fitness

    breed = True
    iteration = 0

    while breed:

        population.create_offspring()
        population.calc_fitness()
        max_fitness = population.max_fitness

        dna_str = ''.join([str(i) for i in population.population[0].dna])

        # Print on a line in terminal, then print over it
        sys.stdout.flush()
        sys.stdout.write("\r")
        sys.stdout.flush()
        sys.stdout.write(f"{iteration:>5} : {max_fitness:.3f} : {dna_str}")

        if iteration == MAX_ITER:
            breed = False

        iteration += 1

    print('\n\n')

    print("\n\nFinished\n")
    
    return population

## Evolve

In [8]:
population = main()


Population Size: 200
Mutation_rate: 0.02
DNA_length: 50
Max_iter : 100

  100 : 0.994 : 11221220000030231012001232123003100232302101113233




Finished



## Run fittest dna sequence of last generation

In [13]:
run_dna(env, population.population[0].dna)

  (Right)
SFFF
FHFH
FFFH
HFFG

 5 steps needed
